In [1]:
import re
import nrrd
import os, glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

import loss
import Helper
import allMetrics

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

physical_devices = tf.config.list_physical_devices('GPU')
try:
  for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [3]:
data_dir = '/home/tester/jianhoong/jh_fyp_work/ct_scans_data/raw_data/'

z_train = os.path.join(data_dir, 'training_data_z')
z_train_image = os.path.join(z_train, 'training_images/training_images')
z_train_mask = os.path.join(z_train, 'training_masks/training_masks')

z_valid = os.path.join(data_dir, 'valid_data_z')
z_valid_image = os.path.join(z_valid, 'valid_images/valid_images')
z_valid_mask = os.path.join(z_valid, 'valid_masks/valid_masks')

z_test = os.path.join(data_dir, 'testing_data_z')
z_test_image = os.path.join(z_test, 'testing_images/testing_images')
z_test_mask = os.path.join(z_test, 'testing_masks/testing_masks')


train_data_with_96 = [15, 16, 20, 21, 23, 33, 37, 52, 64, 69, 72, 76, 77, 83, 92, 94, 95, 97, 103, 116, 123, 129, 157, 159, 162, 163, 170, 171, 180, 191, 192, 198, 200, 207, 209]
valid_data_with_96 = [213, 219, 221, 227, 231, 238, 240, 243, 249, 250, 255, 256, 257, 270]
test_data_with_96 = [271, 272, 274, 277, 283, 286, 287, 293, 296, 300, 303, 305, 306, 314]

train_path = Helper.sorted_alnum([os.path.join(z_train_image, file) for file in os.listdir(z_train_image)  if int(re.findall(r'\d+', file)[0]) in train_data_with_96])
train_mask_path = Helper.sorted_alnum([os.path.join(z_train_mask, file) for file in os.listdir(z_train_mask)  if int(re.findall(r'\d+', file)[0]) in train_data_with_96])

valid_path = Helper.sorted_alnum([os.path.join(z_valid_image, file) for file in os.listdir(z_valid_image)  if int(re.findall(r'\d+', file)[0]) in valid_data_with_96])
valid_mask_path = Helper.sorted_alnum([os.path.join(z_valid_mask, file) for file in os.listdir(z_valid_mask)  if int(re.findall(r'\d+', file)[0]) in valid_data_with_96])

test_path = Helper.sorted_alnum([os.path.join(z_test_image, file) for file in os.listdir(z_test_image)  if int(re.findall(r'\d+', file)[0]) in test_data_with_96])
test_mask_path = Helper.sorted_alnum([os.path.join(z_test_mask, file) for file in os.listdir(z_test_mask)  if int(re.findall(r'\d+', file)[0]) in test_data_with_96])

FileNotFoundError: [Errno 2] No such file or directory: '/home/tester/jianhoong/jh_fyp_work/ct_scans_data/raw_data/training_data_z/training_images/training_images'

In [4]:
model_checkpoint_path = '/home/tester/jianhoong/jh_fyp_work/3D_UNet/Final/SavedModels/Model7.hdf5'

model = tf.keras.models.load_model(model_checkpoint_path, custom_objects = {
                                                                                   'dice_coef':allMetrics.dice_coef,
                                                                                   'tversky_crossentropy':loss.tversky_crossentropy})

2023-01-26 22:45:22.698995: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-26 22:45:23.098738: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13777 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5


In [39]:
input_shape = (64,64,96)
num_class = 1
desired_step = 32

test_patient_img = Helper.process_scan(train_path[3])
test_patient_msk = Helper.read_nrrd_file(train_mask_path[3])

test_patch = patchify(test_patient_img, input_shape, step = desired_step) # Yield 15 x 15 x 3 of size 64 x 64 x 32 cubes
test_mask_patch = patchify(test_patient_msk, input_shape, step = desired_step)

test_patient_cubes = np.reshape(test_patch, (-1, test_patch.shape[3], test_patch.shape[4], test_patch.shape[5])) # Collapse 15 x 15 x 3 into 675 cubes x 64 x 64 x 32
test_patient_mask_cubes = np.reshape(test_mask_patch, (-1, test_mask_patch.shape[3], test_mask_patch.shape[4], test_mask_patch.shape[5])) # 675 x 64 x 64 x 32

test_patient_input = np.expand_dims(test_patient_cubes, axis=4)
test_patient_mask_input = np.expand_dims(test_patient_mask_cubes, axis=4)

test_patient_pred = model.predict(test_patient_input)

test_patient_pred_reshaped = np.reshape(test_patient_pred, 
                                        (test_patch.shape[0], test_patch.shape[1], test_patch.shape[2],
                                         test_patch.shape[3], test_patch.shape[4], test_patch.shape[5]))

reconstruct_pred = unpatchify(test_patient_pred_reshaped, test_patient_img.shape)
# reconstruct_pred[reconstruct_pred < 45] = 0
# reconstruct_pred[reconstruct_pred > 50] = 255

@interact
def explore_prediction(layer = (0,96), view = ["axial"]):
    if view == 'axial':
        img_array_view = test_patient_img[:, :, layer]
        msk_array_view = test_patient_msk[:, :, layer]
        pred_array_view = reconstruct_pred[:, :, layer]
# , cmap='gray'
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_array_view, cmap = "gray", aspect = "auto")
    plt.title('Img', fontsize=10)
    plt.subplot(1, 3, 2)
    plt.imshow(msk_array_view, cmap = "gray",aspect = "auto")
    plt.title('Mask', fontsize=10)
    plt.subplot(1, 3, 3)
    plt.imshow(pred_array_view, cmap = "gray",aspect = "auto")
    plt.title('Prediction', fontsize=10)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=48, description='layer', max=96), Dropdown(description='view', options=(…

In [30]:
@interact
def explore_prediction(layer = (0,511), view = ["coronal"]):
    if view == 'coronal':
        img_array_view = test_patient_img[:, layer, :]
        msk_array_view = test_patient_msk[:, layer, :]
        pred_array_view = reconstruct_pred[:, layer, :]
# , cmap='gray'
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_array_view, cmap = "gray", aspect = "auto")
    plt.title('Img', fontsize=10)
    plt.subplot(1, 3, 2)
    plt.imshow(msk_array_view, cmap = "gray",aspect = "auto")
    plt.title('Mask', fontsize=10)
    plt.subplot(1, 3, 3)
    plt.imshow(pred_array_view, cmap = "gray",aspect = "auto")
    plt.title('Prediction', fontsize=10)
    plt.axis('off')
    plt.show()
    
@interact
def explore_prediction(layer = (0,511), view = ["sagittal"]):
    if view == 'sagittal':
        img_array_view = test_patient_img[layer, :, :]
        msk_array_view = test_patient_msk[layer, :, :]
        pred_array_view = reconstruct_pred[layer, : , :]
# , cmap='gray'
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(img_array_view, cmap = "gray", aspect = "auto")
    plt.title('Img', fontsize=10)
    plt.subplot(1, 3, 2)
    plt.imshow(msk_array_view, cmap = "gray",aspect = "auto")
    plt.title('Mask', fontsize=10)
    plt.subplot(1, 3, 3)
    plt.imshow(pred_array_view, cmap = "gray",aspect = "auto")
    plt.title('Prediction', fontsize=10)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=255, description='layer', max=511), Dropdown(description='view', options…

interactive(children=(IntSlider(value=255, description='layer', max=511), Dropdown(description='view', options…

In [40]:
headerInfo = nrrd.read_header(train_path[3])

In [41]:
pred_filename = "/home/tester/jianhoong/jh_fyp_work/ct_scans_data/prediction_data/P21_pred_Model7.nrrd"
nrrd.write(pred_filename, reconstruct_pred, headerInfo)